<a href="https://colab.research.google.com/github/sharath0899/hds5210/blob/main/Copy_of_midterm_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [ ]:
import requests

def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    """
    Calculate the PRIEST COVID-19 Clinical Severity Score based on various parameters.

    Args:
        sex (str): 'male' or 'female'
        age (int): Age in years
        respiratory_rate (int): Respiratory rate in breaths per minute
        oxygen_saturation (float): Oxygen saturation as a percentage between 0 and 1
        heart_rate (int): Heart rate in beats per minute
        systolic_bp (int): Systolic blood pressure in mmHg
        temperature (float): Body temperature in degrees Celsius
        alertness (str): 'alert' or 'confused/not alert'
        inspired_oxygen (str): 'air' or 'supplemental oxygen'
        performance_status (str): Performance status as described ('unrestricted', 'limited strenuous', etc.)

    Returns:
        float: Risk probability as a number between 0 and 1, or None for invalid inputs.

    Examples:
        >>> priest('male', 55, 22, 0.94, 100, 120, 37.0, 'alert', 'air', 'Unrestricted normal activity')
        0.18
        >>> priest('female', 70, 25, 0.92, 135, 80, 36.5, 'confused or not alert', 'supplemental oxygen', 'Bed/chair bound, no self-care')
        0.59
        >>> priest('male', 85, 18, 0.96, 85, 100, 38.5, 'alert', 'air', 'limited activity, can self-care')
        0.29
        >>> priest('MaLe', 60, 19, 0.40, 75, 120, 20, 'confused or not alert', 'AiR', 'limited Strenuous activity, can do light activity')
        0.46
    """

    # Normalize string inputs
    sex = sex.lower()
    alertness = alertness.lower()
    inspired_oxygen = inspired_oxygen.lower()
    performance_status = performance_status.lower()

    # Validate sex
    if sex not in ['male', 'female']:
        return None

    # Sex score
    score = 1 if sex == 'male' else 0

    # Age score
    if age > 80:
        score += 4
    elif age >= 66 and age <= 80:
        score += 3
    elif age >= 50 and age <=65:
        score += 2
    elif age >= 16 and age <= 49:
        score += 0
    elif age < 16:
        return None

    # Respiratory rate score
    if respiratory_rate > 24:
        score += 3
    elif respiratory_rate >= 21 and respiratory_rate <= 24:
        score += 2
    elif respiratory_rate >= 12 and respiratory_rate <= 20:
        score += 0
    elif respiratory_rate >= 9 and respiratory_rate <= 11:
        score += 1
    elif respiratory_rate < 9:
        score += 3

    # Oxygen saturation score (as a fraction of 1)
    if oxygen_saturation > 0.95:
        score += 0
    elif oxygen_saturation >= 0.94 and oxygen_saturation <= 0.95:
        score += 1
    elif oxygen_saturation >= 0.92 and oxygen_saturation <= 0.93:
        score += 2
    elif oxygen_saturation < 0.92:
        score += 3

    # Heart rate score
    if heart_rate > 130:
        score += 3
    elif heart_rate >= 111 and heart_rate <= 130:
        score += 2
    elif heart_rate >= 91 and heart_rate <= 110:
        score += 1
    elif heart_rate >= 51 and heart_rate <= 90:
        score += 0
    elif heart_rate >= 41 and heart_rate <=50:
        score += 1
    elif heart_rate < 41:
        score += 3

    # Systolic BP score
    if systolic_bp > 219:
        score += 3
    elif systolic_bp >= 111 and systolic_bp <= 219:
        score += 0
    elif systolic_bp >= 101 and systolic_bp <= 110:
        score += 1
    elif systolic_bp >= 91 and systolic_bp <= 100:
        score += 2
    elif systolic_bp < 91:
        score += 3

    # Temperature score
    if temperature > 39.0:
        score += 2
    elif temperature >= 38.1 and temperature <= 39.0:
        score += 1
    elif temperature >= 36.1 and temperature <= 38.0 :
        score += 0
    elif temperature >= 35.1 and temperature <= 36.0:
        score += 1
    elif temperature < 35.1:
        score += 3

    # Alertness score
    if alertness == 'confused or not alert':
        score += 3
    elif alertness == 'alert':
        score += 0
    else:
        return None

    # Inspired oxygen score
    if inspired_oxygen == 'supplemental oxygen':
        score += 2
    elif inspired_oxygen == 'air':
        score += 0
    else:
        return None

    # Performance status score
    if performance_status == 'bed/chair bound, no self-care':
        score += 4
    elif performance_status == 'limited self-care':
        score += 3
    elif performance_status == 'limited activity, can self-care' :
        score += 2
    elif performance_status == 'limited strenuous activity, can do light activity':
        score += 1
    elif performance_status == 'unrestricted normal activity':
        score += 0
    else:
        return None

    # Translate score to risk probability
    if score >= 26:
        return 0.99
    elif score >= 17 and score <= 25:
        return 0.59
    else:
        risk_mapping = {
            0: 0.01, 1: 0.01, 2: 0.02, 3: 0.02, 4: 0.03, 5: 0.09, 6: 0.15, 7: 0.18,
            8: 0.22, 9: 0.26, 10: 0.29, 11: 0.34, 12: 0.38, 13: 0.46, 14: 0.47,
            15: 0.49, 16: 0.55
        }
        return risk_mapping.get(score, None)


In [ ]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/doctest.py", line 1501, in run
    sys.settrace(save_trace)



Finding tests in NoName
Trying:
    priest('male', 55, 22, 0.94, 100, 120, 37.0, 'alert', 'air', 'Unrestricted normal activity')
Expecting:
    0.18
ok
Trying:
    priest('female', 70, 25, 0.92, 135, 80, 36.5, 'confused or not alert', 'supplemental oxygen', 'Bed/chair bound, no self-care')
Expecting:
    0.59
ok
Trying:
    priest('male', 85, 18, 0.96, 85, 100, 38.5, 'alert', 'air', 'limited activity, can self-care')
Expecting:
    0.29
ok
Trying:
    priest('MaLe', 60, 19, 0.40, 75, 120, 20, 'confused or not alert', 'AiR', 'limited Strenuous activity, can do light activity')
Expecting:
    0.46
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [ ]:
import requests

def find_hospital(age, sex, risk_pct):
    """
    Finds a hospital for a patient based on their age, sex, and risk percentage.

    This function calls a REST API to determine which hospital the patient should be sent to.

    Args:
        age (int): The patient's age in years.
        sex (str): The patient's sex ('male' or 'female').
        risk_pct (float): The patient's risk percentage as a number between 0 and 1.

    Returns:
        str: The name of the hospital, or None if the request fails or input is invalid.

    Examples:
        >>> find_hospital(40, 'male', 0.1)
        'Southwest Hospital and Medical Center'
        >>> find_hospital(70, 'female', 0.9)
        'Wesley Woods Geriatric Hospital'
        >>> find_hospital(30, 'MALE', 0.3)  # Case-insensitive input

    """

    # Normalize the sex input (convert to lowercase)
    # sex = sex.lower()

    # Validate sex to make sure it's either 'male' or 'female'
    if sex not in ['male', 'female']:
        return None  # Return None if the sex is invalid

    # Construct the API URL with the patient's data
    url = f"https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk_pct}"

    try:
        # Make a GET request to the API
        response = requests.get(url)

        # Check if the response is successful (status code 200)
        if response.status_code != 200:
            return None  # Return None if there is an error in the request

        # Parse the JSON response from the API
        data = response.json()

        # Extract and return the hospital name from the JSON data
        return data.get('hospital')

    except Exception:
        # Return None if there's an error (like a connection issue or invalid JSON)
        return None



In [ ]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(70, 'female', 0.9)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(30, 'MALE', 0.3)  # Case-insensitive input
Expecting nothing
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [ ]:
import requests

def get_address(hospital_name):
    """
    Looks up the address of a hospital based on its name from a JSON file.

    Args:
        hospital_name (str): The name of the hospital to search for.

    Returns:
        str: The address of the hospital, or None if not found.

    Example:
        >>> get_address('Grady Memorial Hospital')
        '80 JESSE HILL, JR DRIVE SE'
        >>> get_address('Piedmont Hospital')
        '1968 PEACHTREE ROAD NW'
        >>> get_address('Unknown Hospital')  # Non-existent hospital
    """

    # URL of the JSON file with hospital data
    url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"

    try:
        # Fetch the JSON file from the URL
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code != 200:
            return None

        # Parse the JSON data
        hospitals_data = response.json()
        req_hospital_name = hospitals_data.get(hospital_name.upper())
        if req_hospital_name:
            return req_hospital_name.get('ADDRESS')

        return None

        # Search for the hospital by name in the JSON data
        #for hospital in hospitals_data:
         #   if hospital.lower() == hospital_name.lower():  # Case-insensitive match
          #      return hospital['ADDRESS']  # Return the address if the hospital is found

        # If no match was found, return None
        #return None

    except Exception:
        # In case of any error (network issue, invalid JSON, etc.), return None
        return None


In [ ]:
# Test case 1: Grady Memorial Hospital
print(get_address('Grady Memorial Hospital'))
# Output: '80 Jesse Hill Jr Drive SE, Atlanta, GA'

# Test case 2: Piedmont Hospital
print(get_address('Piedmont Hospital'))
# Output: '1968 Peachtree Road NW, Atlanta, GA'

# Test case 3: Non-existent hospital
print(get_address('Unknown Hospital'))
# Output: None


80 JESSE HILL, JR DRIVE SE
1968 PEACHTREE ROAD NW
None


In [ ]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)


Finding tests in NoName
Trying:
    get_address('Grady Memorial Hospital')
Expecting:
    '80 JESSE HILL, JR DRIVE SE'
ok
Trying:
    get_address('Piedmont Hospital')
Expecting:
    '1968 PEACHTREE ROAD NW'
ok
Trying:
    get_address('Unknown Hospital')  # Non-existent hospital
Expecting nothing
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [ ]:
import requests
import csv
import json

def process_people(file_url):
    """
    Processes a population from a pipe-delimited file (PSV), calculates the PRIEST risk,
    determines the hospital for each patient, and retrieves the hospital's address.

    Args:
        file_url (str): The URL of the pipe-delimited file containing patient data.

    Returns:
        list: A list of dictionaries, where each dictionary contains patient data in the format:
              { patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, risk, hospital, address] }

    Example:
        >>> process_people("https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh")

    """

    # Download the people.psv file from the given URL
    response = requests.get(file_url)
    if response.status_code != 200:
        return None  # If the download fails, return None

    # Decode the file content and split it into lines
    file_content = response.content.decode('utf-8')
    lines = file_content.splitlines()

    # Initialize the result list
    result = []

    # Use csv.reader to read the pipe-delimited file
    reader = csv.reader(lines, delimiter='|')

    # Skip the header row
    next(reader)

    for row in reader:
        # Extract the patient data from the row
        #print(row)
        patient_id = row[0]
        sex = row[1]
        age = int(row[2])
        breath = int(row[3])
        o2sat = float(row[4])
        heart = int(row[5])
        systolic = int(row[6])
        temp = float(row[7])
        alertness = row[8]
        inspired = row[9]
        status = row[10]

        sex = sex.lower()
        alertness = alertness.lower()
        inspired = inspired.lower()
        status = status.lower()

        # Step 1: Calculate the PRIEST risk score
        risk = priest(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status)
        if risk is None:
            continue  # If the risk couldn't be calculated, skip this patient
        #print(row)
        # Step 2: Determine the hospital based on the risk, age, and sex
        hospital = find_hospital(age, sex, risk)
        if hospital is None:
            continue  # If the hospital couldn't be found, skip this patient
        #print(row)
        # Step 3: Get the hospital address
        address = get_address(hospital)
        if address is None:
            continue  # If the address couldn't be found, skip this patient
        #print(row)
        # Step 4: Append the result for this patient
        result.append({patient_id : [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, risk, hospital, address]})


    #return json.dumps(result)
    return result


In [ ]:
import doctest
doctest.run_docstring_examples(process_people, globals(),verbose=True)


Finding tests in NoName
Trying:
    process_people("https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh")
Expecting nothing
**********************************************************************
File "__main__", line 18, in NoName
Failed example:
    process_people("https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh")
Expected nothing
Got:
    [{'E9559': ['female', 40, 24, 0.96, 105, 115, 34.9, 'alert', 'air', 'unrestricted normal activity', 0.15, 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE']}, {'E9385': ['female', 51, 19, 0.99, 90, 91, 38.1, 'alert', 'supplemental oxygen', 'unrestricted normal activity', 0.18, 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE']}, {'E3067': ['female', 40, 29, 0.96, 105, 95, 38.1, 'alert', 'air', 'unrestricted normal activity', 0.18, 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE']}, {'E9422': ['female', 66, 19, 0.96, 135, 1

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [ ]:
"""
import requests
import json

# Step 1: Download the people_results.json file
results_url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"
response = requests.get(results_url)
if response.status_code == 200:
    # Load the expected results from the downloaded JSON
    expected_results = response.json()
else:
    print("Failed to download people_results.json.")
    expected_results = None

# Step 2: Run the process_people() function to get your results
people_url = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"
my_results = process_people(people_url)

# Step 3: Compare the results
if expected_results is not None:
    # Check if the lengths of the results match
    if len(my_results) == len(expected_results):
        print(f"Both results have the same number of records: {len(my_results)}")

        # Compare each patient's record
        for i, (my_record, expected_record) in enumerate(zip(my_results, expected_results), start=1):
            if my_record != expected_record:
                print(f"Difference found in patient {i}:")
                print("Your result: ", my_record)
                print("Expected result: ", expected_record)
            else:
                print(f"Patient {i} matches.")
    else:
        print(f"Results have different lengths: {len(my_results)} vs {len(expected_results)}")
else:
    print("No expected results to compare against.")

"""

import requests
import json

# Step 1: Download the people_results.json file
results_url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"
response = requests.get(results_url)
count = 0
if response.status_code == 200:
    # Load the expected results from the downloaded JSON
    expected_results = response.json()
else:
    print("Failed to download people_results.json.")
    expected_results = None

# Step 2: Run the process_people() function to get your results
people_url = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"
my_results = process_people(people_url)

# Step 3: Compare the results
if expected_results is not None:
    # Check if the lengths of the results match
    if len(my_results) == len(expected_results):
        print(f"Both results have the same number of records: {len(my_results)}")
        # Initialize an empty list to store the converted entries
        expected_list = []
        result_list = []

        # Iterate through the JSON data and extract the values
        for key, values in expected_results.items():
          # Looping through values to convert str to lower case if needed
                for i in range(len(values)):
                    if isinstance(values[i], str):
                        values[i] = values[i].lower()
                expected_list.append( [key] +values)
        #print(expected_list)

        for record in my_results:
            for key, values in record.items():
                # Looping through values to convert str to lower case if needed
                for i in range(len(values)):
                    if isinstance(values[i], str):
                        values[i] = values[i].lower()
            result_list.append([key] + values)
        #print(result_list)

       # Compare each patient's record
        for i, (my_record, expected_record) in enumerate(zip(result_list, expected_list), start=1):
            if my_record != expected_record:
                print(f"Difference found in patient {i}:")
                print("Your result: ", my_record)
                print("Expected result: ", expected_record)
            else:
                #print(f"Patient {my_record[0]} matches.")
                count += 1
        print(f"{count} patients matched")


    else:
        print(f"Results have different lengths: {len(my_results)} vs {len(expected_results)}")

else:
    print("No expected results to compare against.")


Both results have the same number of records: 99
99 patients matched


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---